Spark learning path

In [2]:
import pyspark

from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("sparkapp").getOrCreate()

from pyspark.sql.functions import lit, col,sum,when,mode,trim
from pyspark.ml.feature import Imputer,StringIndexer

24/09/09 09:42:19 WARN Utils: Your hostname, AI-CJB-LAP-459 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface wlp0s20f3)
24/09/09 09:42:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 09:42:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [125]:

df = spark.read.option("header","true").csv("/home/ai/Documents/sampleDataset.csv")

df.show()

df = df.filter(col("name").isNotNull())
df.show()

+---+----+-----+
| id|name|marks|
+---+----+-----+
|  1|  A1|   55|
|  2|  A2|   66|
|  3|  A3|   77|
+---+----+-----+

+---+----+-----+
| id|name|marks|
+---+----+-----+
|  1|  A1|   55|
|  2|  A2|   66|
|  3|  A3|   77|
+---+----+-----+



In [31]:
data = [(('James','','Smith'),'1991-04-01','M',3000),
  (('Michael','Rose',''),'2000-05-19','M',4000),
  (('Robert','','Williams'),'1978-09-05','M',4000),
  (('Maria','Anne','Jones'),'1967-12-01','F',4000),
  (('Jen','Mary','Brown'),'1980-02-17','F',-1)
]

In [18]:
filtered_df = df.filter(col("Gender").isNull())

filtered_df.show()


+----+---+------+------+
|Name|Age|Gender|Course|
+----+---+------+------+
+----+---+------+------+



In [98]:
# df = df.withColumn("Gender",trim(col("Gender")))
# # df = df.filter(df['Gender'] == '')
# # empty_gender_df.show()
# df.show()
# print(df.count())

+------+---+------+------+
|  Name|Age|Gender|Course|
+------+---+------+------+
|dhivya| 20|      |   EEE|
+------+---+------+------+

1


In [19]:
df= df.withColumn(
    'Gender',
    when(col('Gender') .isin(["",' ',"? ","~"] ),None).otherwise(col('Gender'))
)
df.show()

+---------+---+------+------+
|     Name|Age|Gender|Course|
+---------+---+------+------+
|   dhivya| 20|  NULL|   EEE|
|   dhivya| 20|  NULL|   EEE|
|  sowmiya| 21|     F|   ECE|
|     guru| 22|     M|   CSE|
|santhusta| 20|     F| AI&ML|
|   Janani| 21|     F| AI&DS|
+---------+---+------+------+



In [20]:
filtered_df = df.filter(col("Gender").isNull())

filtered_df.show()

+------+---+------+------+
|  Name|Age|Gender|Course|
+------+---+------+------+
|dhivya| 20|  NULL|   EEE|
|dhivya| 20|  NULL|   EEE|
+------+---+------+------+



In [21]:
mode_row = df.groupBy('Gender').count().orderBy('count', ascending=False).first()
if mode_row:
    mode_gender = mode_row['Gender']  
else:
    mode_gender = None
    
print(mode_row)

Row(Gender='F', count=3)


In [22]:
# mode_gender = df.groupBy('Gender').count().orderBy('count', ascending=False).first()[0]
# print(mode_gender)
df=df.na.fill({'Gender': mode_gender})
print(mode_gender)
df.show()

F
+---------+---+------+------+
|     Name|Age|Gender|Course|
+---------+---+------+------+
|   dhivya| 20|     F|   EEE|
|   dhivya| 20|     F|   EEE|
|  sowmiya| 21|     F|   ECE|
|     guru| 22|     M|   CSE|
|santhusta| 20|     F| AI&ML|
|   Janani| 21|     F| AI&DS|
+---------+---+------+------+



In [23]:
indexer = StringIndexer(inputCol="Gender", outputCol="Gender_indexed")
df_indexed = indexer.fit(df).transform(df)

In [24]:
indexer = StringIndexer(inputCol="Gender", outputCol="Gender_indexed")
df_indexed = indexer.fit(df).transform(df)

# Imputing the indexed column
imputer = Imputer(inputCols=["Gender_indexed"], outputCols=["Gender_imputed"]).setStrategy("mode")
model = imputer.fit(df_indexed)
df_imputed = model.transform(df_indexed)
df.show()

+---------+---+------+------+
|     Name|Age|Gender|Course|
+---------+---+------+------+
|   dhivya| 20|     F|   EEE|
|   dhivya| 20|     F|   EEE|
|  sowmiya| 21|     F|   ECE|
|     guru| 22|     M|   CSE|
|santhusta| 20|     F| AI&ML|
|   Janani| 21|     F| AI&DS|
+---------+---+------+------+



base schema with nested structure

In [27]:
from pyspark.sql.types import StructType, StructField,StringType, IntegerType
schema = StructType([StructField("name",StructType([
        StructField("firstname", StringType(),True),
        
        StructField("lastname", StringType(),True)
    ])),
    StructField("dob", StringType(),True),
    StructField("gender", StringType(),True),
    StructField("gender",IntegerType(),True)
])